In [1]:
import krippendorff
import numpy as np
import pandas as pd
import re
import pickle
import random
import json
from collections import Counter
from sklearn.metrics import f1_score
from itertools import combinations
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score


In [2]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")

In [3]:
classes_5 = ['Unrelated', 'Consistent', 'Indirect inconsistency', "Factual inconsistency", 'Surface contradiction']                             
classes_3 = ['Unrelated', 'Consistent', 'Inconsistent']
classes_5_casefold = classes_5 + ["Surface Contradiction", "Factual Inconsistency", "Indirect Inconsistency"]
classes_5_to_3 = {'Unrelated': 'Unrelated', 'Consistent': 'Consistent', 'Indirect inconsistency': 'Inconsistent', "Factual inconsistency": 'Inconsistent', 'Surface contradiction': 'Inconsistent'}

In [4]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\bootstrap_predictions_tuples.pkl", 'rb') as f:
    bootstrap_predictions_tuples = pickle.load(f)

### Calculate confidence for each model

In [4]:
df_gold['']

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,Model_agreement_with_human,model_agreement_with_human_3_classes,ChatGPT-3.5 5 classes,...,Llama 70B majority,ChatGPT-3.5 5 runs,ChatGPT-4 5 runs,ChatGPT-3.5 majority,ChatGPT-4 majority,Llama 8B majority 3 class,Llama 70B majority 3 class,ChatGPT-3.5 majority 3 class,ChatGPT-4 majority 3 class,bart_NLI
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,1.000000,Factual inconsistency,...,Indirect inconsistency,"['Factual inconsistency', 'Surface contradicti...","['Indirect inconsistency', 'Indirect inconsist...",Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Inconsistent
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,0.800000,Surface contradiction,...,Indirect inconsistency,"['Surface contradiction', 'Factual inconsisten...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Consistent
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,1.000000,Factual inconsistency,...,Indirect inconsistency,"['Factual inconsistency', 'Factual inconsisten...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Inconsistent
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,Indirect inconsistency,Unrelated,0.200000,0.600000,Indirect inconsistency,...,Indirect inconsistency,"['Indirect inconsistency', 'Indirect inconsist...","['Unrelated', 'Unrelated', 'Unrelated', 'Unrel...",Indirect inconsistency,Unrelated,Consistent,Inconsistent,Inconsistent,Unrelated,Inconsistent
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent,Indirect inconsistency,Inconsistent,0.833333,0.833333,Factual inconsistency,...,Indirect inconsistency,"['Factual inconsistency', 'Factual inconsisten...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,Consistent,Inconsistent,Inconsistent,Inconsistent,Inconsistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Inconsistent,0.400000,1.000000,Surface contradiction,...,Factual inconsistency,"['Surface contradiction', 'Factual inconsisten...","['Indirect inconsistency', 'Indirect inconsist...",Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Consistent
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,1.000000,Surface contradiction,...,Indirect inconsistency,"['Surface contradiction', 'Surface contradicti...","['Indirect inconsistency', 'Indirect inconsist...",Surface contradiction,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Inconsistent
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,0.800000,Surface contradiction,...,Consistent,"['Surface contradiction', 'Surface contradicti...","['Consistent', 

In [15]:
counter = Counter(eval(df_gold['ChatGPT-4 5 runs'][324]))

In [16]:
counter

Counter({'Unrelated': 4, 'Consistent': 1})

In [18]:
most_common_element, count = counter.most_common(1)[0]

In [19]:
count

4

In [22]:
round(count/(len(eval(df_gold['ChatGPT-4 5 runs'][324])))*100, 3)

80.0

In [17]:
df_gold['ChatGPT-4 Confidence'] = 0
df_gold['ChatGPT-3.5 Confidence'] = 0
df_gold['LLaMA 8B Confidence'] = 0
df_gold['LLaMA 70B Confidence'] = 0

In [30]:
df_gold

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,Model_agreement_with_human,model_agreement_with_human_3_classes,ChatGPT-3.5 5 classes,...,ChatGPT-4 majority,Llama 8B majority 3 class,Llama 70B majority 3 class,ChatGPT-3.5 majority 3 class,ChatGPT-4 majority 3 class,bart_NLI,ChatGPT-4 Confidence,ChatGPT-3.5 Confidence,LLaMA 8B Confidence,LLaMA 70B Confidence
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,1.000000,Factual inconsistency,...,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,80,80,100
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,0.800000,Surface contradiction,...,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Consistent,100,60,80,100
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,1.000000,Factual inconsistency,...,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,100,80,100
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,Indirect inconsistency,Unrelated,0.200000,0.600000,Indirect inconsistency,...,Unrelated,Consistent,Inconsistent,Inconsistent,Unrelated,Inconsistent,100,80,80,100
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent,Indirect inconsistency,Inconsistent,0.833333,0.833333,Factual inconsistency,...,Indirect inconsistency,Consistent,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,80,80,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Inconsistent,0.400000,1.000000,Surface contradiction,...,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Consistent,80,60,60,80
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,1.000000,Surface contradiction,...,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Inconsistent,100,100,80,100
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,0.800000,Surface contradiction,...,Consistent,Consistent,Consistent,Inconsistent,Consistent,Inconsistent,100,80,60,100
696,Text 1: The level of social basic security mus...,Factual inconsistency,40.000,"['Indirect inconsistency', 'Factual inconsiste...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,0.600000,Surface contradiction,...,Indirect inconsistency,Inconsistent,Inconsistent,Inconsistent,Inconsistent,Consistent,80,60,60,80


In [29]:
for i in range(len(df_gold)):
    counter = Counter(eval(df_gold['LLaMA 70B 5 runs'][i]))
    most_common_element, count = counter.most_common(1)[0]
    confidence_score = round(count/(len(eval(df_gold['LLaMA 70B 5 runs'][i])))*100, 3)
    df_gold['LLaMA 70B Confidence'][i] = confidence_score

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_274840\2911243292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['LLaMA 70B Confidence'][i] = confidence_score
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_274840\2911243292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['LLaMA 70B Confidence'][i] = confidence_score
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_274840\2911243292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

### Bootstrap 3 classes

In [17]:
bootstrap_predictions_tuples_3_classes = []

for i in range(len(df_gold)):
    # we have 5 tuples per sample 
    strings = eval(df_gold['All answers'][i])
    # create N-1 tuples
    versions = [(strings[j], strings[:j] + strings[j+1:]) for j in range(len(strings))]
    # print(len(versions))
    # go through each element, value is held out "ground truth"
    for value, version in versions:
        list_annotator_dropped = [classes_5_to_3[el] for el in version]
        version = [classes_5_to_3[el] for el in version]
        value = classes_5_to_3[value]
        counter = Counter(list_annotator_dropped)
        # identify the majority vote and sample randomly if more than 1
        max_count = max(counter.values())
        top_labels = [label for label, count in counter.items() if count == max_count]
        # this is today's ground truth
        majority = random.choice(top_labels)
        # 10 bootstraps of the same pool where we took the majority from 
        bootstraps = [random.choices(version, k=len(version)) for _ in range(10)]
        bootstrap_majorities = []
        for bootstrap in bootstraps:
            counter = Counter(bootstrap)
            max_count = max(counter.values())
            top_labels = [label for label, count in counter.items() if count == max_count]
            # this is tomorrow's ground truth
            tomorrow_majority = random.choice(top_labels)
            bootstrap_majorities.append(tomorrow_majority)
        # print(len(bootstrap_majorities))
        to_append = []
        for tomorrow_majority in bootstrap_majorities:
            to_append.append((majority, tomorrow_majority))
        bootstrap_predictions_tuples_3_classes.extend(to_append)        
            


      

In [18]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\bootstrap_predictions_tuples_3_classes.pkl', 'wb') as f:
    pickle.dump(bootstrap_predictions_tuples_3_classes, f)

### Recalculate F1-score for majority LLM - 3 classes

In [23]:
f1_score_per_model_type_majority_LLM_3_classes = dict()


for model_type in prediction_ground_truth_tuples_majority_LLM_3_classes:
    f1_score_per_model_type_majority_LLM_3_classes[model_type] = dict()
    print(model_type)
    true_labels = []
    predictions = []
    for ground_truth, prediction in prediction_ground_truth_tuples_majority_LLM_3_classes[model_type]:
        true_labels.append(ground_truth)
        predictions.append(prediction)
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_f1_score = f1_score(true_labels, predictions, labels=classes_3, average=None)
    print("Per class", res_f1_score)
    my_dict = dict(zip(classes_3, res_f1_score))
    f1_score_per_model_type_majority_LLM_3_classes[model_type] = my_dict
    print('Just mean', np.mean(res_f1_score))
    res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_3, average='macro')
    print("Macro", res_f1_score_macro)
    f1_score_per_model_type_majority_LLM_3_classes[model_type]['macro'] = res_f1_score_macro

    res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_3, average='micro')
    print("Micro", res_f1_score_micro)
    f1_score_per_model_type_majority_LLM_3_classes[model_type]['micro'] = res_f1_score_micro


Humans
Per class [0.58319039 0.68117519 0.88539778]
Just mean 0.716587789808988
Macro 0.716587789808988
Micro 0.8087943262411349
ChatGPT-4
Per class [0.62       0.7026455  0.87325702]
Just mean 0.731967508859635
Macro 0.731967508859635
Micro 0.8036879432624113
ChatGPT-3.5
Per class [0.1986755  0.41463415 0.86750681]
Just mean 0.49360548500643536
Macro 0.49360548500643536
Micro 0.7733333333333333
LLaMA 8B
Per class [0.21551724 0.34756554 0.6726674 ]
Just mean 0.41191672763789944
Macro 0.41191672763789944
Micro 0.535886524822695
LLaMA 70B
Per class [0.55253837 0.74193548 0.90612555]
Just mean 0.7335331332738586
Macro 0.7335331332738586
Micro 0.8419858156028368


In [25]:
model_type = 'bootstrap'
f1_score_per_model_type_majority_LLM_3_classes['bootstrap'] = dict()
true_labels = []
predictions = []
for ground_truth, prediction in bootstrap_predictions_tuples_3_classes:
    true_labels.append(ground_truth)
    predictions.append(prediction)
res_f1_score = f1_score(true_labels, predictions, labels=classes_3, average=None)
print("Per class", res_f1_score)
my_dict = dict(zip(classes_3, res_f1_score))
f1_score_per_model_type_majority_LLM_3_classes[model_type] = my_dict
print('Just mean', np.mean(res_f1_score))
res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_3, average='macro')
print("Macro", res_f1_score_macro)
f1_score_per_model_type_majority_LLM_3_classes[model_type]['macro'] = res_f1_score_macro

res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_3, average='micro')
print("Micro", res_f1_score_micro)
f1_score_per_model_type_majority_LLM_3_classes[model_type]['micro'] = res_f1_score_micro
   

Per class [0.76945039 0.82290398 0.93802598]
Just mean 0.8434601173271709
Macro 0.8434601173271709
Micro 0.8964822695035461


In [26]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\f1_score_per_model_type_majority_LLM_3_classes.pkl', 'wb') as f:
    pickle.dump(f1_score_per_model_type_majority_LLM_3_classes, f)

### Recalculate F1-score for 5 runs - 3 classes

In [19]:
f1_score_per_model_type_5_runs_3_classes = dict()

for model_type in prediction_ground_truth_tuples_5_runs_3_classes:
    f1_score_per_model_type_5_runs_3_classes[model_type] = dict()
    print(model_type)
    true_labels = []
    predictions = []
    for ground_truth, prediction in prediction_ground_truth_tuples_5_runs_3_classes[model_type]:
        true_labels.append(ground_truth)
        predictions.append(prediction)
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_f1_score = f1_score(true_labels, predictions, labels=classes_3, average=None)
    print("Per class", res_f1_score)
    my_dict = dict(zip(classes_3, res_f1_score))
    f1_score_per_model_type_5_runs_3_classes[model_type] = my_dict
    print('Just mean', np.mean(res_f1_score))
    res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_3, average='macro')
    print("Macro", res_f1_score_macro)
    f1_score_per_model_type_5_runs_3_classes[model_type]['macro'] = res_f1_score_macro

    res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_3, average='micro')
    print("Micro", res_f1_score_micro)
    f1_score_per_model_type_5_runs_3_classes[model_type]['micro'] = res_f1_score_micro



Humans
Per class [0.59846547 0.67965368 0.88592772]
Just mean 0.7213489577909499
Macro 0.7213489577909499
Micro 0.8110638297872339
ChatGPT-4
Per class [0.62141872 0.67434073 0.86991021]
Just mean 0.7218898871563099
Macro 0.7218898871563099
Micro 0.7973900709219858
ChatGPT-3.5
Per class [0.18151815 0.40425532 0.86419163]
Just mean 0.4833217013589303
Macro 0.4833217013589303
Micro 0.7674893617021277
LLaMA 8B
Per class [0.27590759 0.40278408 0.69757761]
Just mean 0.4587564275219275
Macro 0.4587564275219275
Micro 0.5698156028368795
LLaMA 70B
Per class [0.54583723 0.74070946 0.90266026]
Just mean 0.7297356514187122
Macro 0.7297356514187122
Micro 0.8376170212765958


In [20]:
model_type = 'bootstrap'
f1_score_per_model_type_5_runs_3_classes['bootstrap'] = dict()
true_labels = []
predictions = []
for ground_truth, prediction in bootstrap_predictions_tuples_3_classes:
    true_labels.append(ground_truth)
    predictions.append(prediction)
res_f1_score = f1_score(true_labels, predictions, labels=classes_3, average=None)
print("Per class", res_f1_score)
my_dict = dict(zip(classes_3, res_f1_score))
f1_score_per_model_type_5_runs_3_classes[model_type] = my_dict
print('Just mean', np.mean(res_f1_score))
res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_3, average='macro')
print("Macro", res_f1_score_macro)
f1_score_per_model_type_5_runs_3_classes[model_type]['macro'] = res_f1_score_macro

res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_3, average='micro')
print("Micro", res_f1_score_micro)
f1_score_per_model_type_5_runs_3_classes[model_type]['micro'] = res_f1_score_micro
   

Per class [0.76945039 0.82290398 0.93802598]
Just mean 0.8434601173271709
Macro 0.8434601173271709
Micro 0.8964822695035461


In [22]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\f1_score_per_model_type_5_runs_3_classes.pkl', 'wb') as f:
    pickle.dump(f1_score_per_model_type_5_runs_3_classes, f)

### Recalculate F1-score for 5 runs

In [42]:
f1_score_per_model_type_5_runs = dict()

for model_type in prediction_ground_truth_tuples_5_runs:
    f1_score_per_model_type_5_runs[model_type] = dict()
    print(model_type)
    true_labels = []
    predictions = []
    for ground_truth, prediction in prediction_ground_truth_tuples_5_runs[model_type]:
        true_labels.append(ground_truth)
        predictions.append(prediction)
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_f1_score = f1_score(true_labels, predictions, labels=classes_5, average=None)
    print("Per class", res_f1_score)
    my_dict = dict(zip(classes_5, res_f1_score))
    f1_score_per_model_type_5_runs[model_type] = my_dict
    print('Just mean', np.mean(res_f1_score))
    res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_5, average='macro')
    print("Macro", res_f1_score_macro)
    f1_score_per_model_type_5_runs[model_type]['macro'] = res_f1_score_macro

    res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_5, average='micro')
    print("Micro", res_f1_score_micro)
    f1_score_per_model_type_5_runs[model_type]['micro'] = res_f1_score_micro



Humans
Per class [0.57928803 0.67559217 0.44311377 0.39754413 0.55431591]
Just mean 0.5299708026780671
Macro 0.5299708026780671
Micro 0.5174468085106383
ChatGPT-4
Per class [0.61970184 0.67863555 0.45127693 0.23447591 0.31134367]
Just mean 0.4590867798983247
Macro 0.4590867798983247
Micro 0.4702978723404255
ChatGPT-3.5
Per class [0.16442451 0.38424897 0.24084527 0.35903064 0.41092375]
Just mean 0.31189462912921684
Macro 0.31189462912921684
Micro 0.3357163120567376
LLaMA 8B
Per class [0.29113725 0.41291642 0.16266945 0.24903503 0.27922624]
Just mean 0.2789968781886877
Macro 0.2789968781886877
Micro 0.27914893617021275
LLaMA 70B
Per class [0.53016772 0.7294307  0.509321   0.24245305 0.48415156]
Just mean 0.4991048046691481
Macro 0.4991048046691481
Micro 0.5063829787234042


In [43]:
model_type = 'bootstrap'
f1_score_per_model_type_5_runs['bootstrap'] = dict()
true_labels = []
predictions = []
for ground_truth, prediction in bootstrap_predictions_tuples:
    true_labels.append(ground_truth)
    predictions.append(prediction)
res_f1_score = f1_score(true_labels, predictions, labels=classes_5, average=None)
print("Per class", res_f1_score)
my_dict = dict(zip(classes_5, res_f1_score))
f1_score_per_model_type_5_runs[model_type] = my_dict
print('Just mean', np.mean(res_f1_score))
res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_5, average='macro')
print("Macro", res_f1_score_macro)
f1_score_per_model_type_5_runs[model_type]['macro'] = res_f1_score_macro

res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_5, average='micro')
print("Micro", res_f1_score_micro)
f1_score_per_model_type_5_runs[model_type]['micro'] = res_f1_score_micro
   

Per class [0.76555798 0.81462905 0.69751779 0.6507764  0.7545423 ]
Just mean 0.7366047033160935
Macro 0.7366047033160935
Micro 0.7307801418439717


In [44]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\f1_score_per_model_type_5_runs.pkl', 'wb') as f:
    pickle.dump(f1_score_per_model_type_5_runs, f)

### Recalculate F1-score for majority LLM

In [39]:
f1_score_per_model_type_majority_LLM = dict()


for model_type in prediction_ground_truth_tuples_majority_LLM:
    f1_score_per_model_type_majority_LLM[model_type] = dict()
    print(model_type)
    true_labels = []
    predictions = []
    for ground_truth, prediction in prediction_ground_truth_tuples_majority_LLM[model_type]:
        true_labels.append(ground_truth)
        predictions.append(prediction)
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_f1_score = f1_score(true_labels, predictions, labels=classes_5, average=None)
    print("Per class", res_f1_score)
    my_dict = dict(zip(classes_5, res_f1_score))
    f1_score_per_model_type_majority_LLM[model_type] = my_dict
    print('Just mean', np.mean(res_f1_score))
    res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_5, average='macro')
    print("Macro", res_f1_score_macro)
    f1_score_per_model_type_majority_LLM[model_type]['macro'] = res_f1_score_macro

    res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_5, average='micro')
    print("Micro", res_f1_score_micro)
    f1_score_per_model_type_majority_LLM[model_type]['micro'] = res_f1_score_micro



Humans
Per class [0.5984     0.67708333 0.44347351 0.39227799 0.55892649]
Just mean 0.5340322650203002
Macro 0.5340322650203002
Micro 0.5214184397163121
ChatGPT-4
Per class [0.63294798 0.71399594 0.44950213 0.19768935 0.31588613]
Just mean 0.4620043066357198
Macro 0.4620043066357198
Micro 0.47404255319148936
ChatGPT-3.5
Per class [0.1744186  0.39349593 0.22979985 0.35601118 0.43161634]
Just mean 0.31706838319602587
Macro 0.31706838319602587
Micro 0.3415602836879433
LLaMA 8B
Per class [0.21864952 0.36337209 0.13168087 0.16833333 0.27127385]
Just mean 0.230661932817461
Macro 0.230661932817461
Micro 0.23687943262411348
LLaMA 70B
Per class [0.53061224 0.73738414 0.50658561 0.25298329 0.47294292]
Just mean 0.500101642433283
Macro 0.500101642433283
Micro 0.5049645390070922


In [40]:
model_type = 'bootstrap'
f1_score_per_model_type_majority_LLM['bootstrap'] = dict()
true_labels = []
predictions = []
for ground_truth, prediction in bootstrap_predictions_tuples:
    true_labels.append(ground_truth)
    predictions.append(prediction)
res_f1_score = f1_score(true_labels, predictions, labels=classes_5, average=None)
print("Per class", res_f1_score)
my_dict = dict(zip(classes_5, res_f1_score))
f1_score_per_model_type_majority_LLM[model_type] = my_dict
print('Just mean', np.mean(res_f1_score))
res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_5, average='macro')
print("Macro", res_f1_score_macro)
f1_score_per_model_type_majority_LLM[model_type]['macro'] = res_f1_score_macro

res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_5, average='micro')
print("Micro", res_f1_score_micro)
f1_score_per_model_type_majority_LLM[model_type]['micro'] = res_f1_score_micro
   

Per class [0.76555798 0.81462905 0.69751779 0.6507764  0.7545423 ]
Just mean 0.7366047033160935
Macro 0.7366047033160935
Micro 0.7307801418439717


In [41]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\f1_score_per_model_type_majority_LLM.pkl', 'wb') as f:
    pickle.dump(f1_score_per_model_type_majority_LLM, f)

### Recalculate precision for 5 runs - 3 classes

In [58]:
precision_per_model_type_5_runs_3_classes = dict()
for model_type in prediction_ground_truth_tuples_5_runs_3_classes:
    precision_per_model_type_5_runs_3_classes[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples_5_runs_3_classes[model_type]:
        try:
            precision_per_model_type_5_runs_3_classes[model_type][prediction]['predicted'] +=1 
            if ground_truth == prediction:
                precision_per_model_type_5_runs_3_classes[model_type][prediction]['actual'] +=1 
        except KeyError:
            precision_per_model_type_5_runs_3_classes[model_type][prediction] = dict() 
            precision_per_model_type_5_runs_3_classes[model_type][prediction]['predicted'] = 1
            precision_per_model_type_5_runs_3_classes[model_type][prediction]['actual'] = 0
            if ground_truth == prediction:
                precision_per_model_type_5_runs_3_classes[model_type][prediction]['actual'] +=1 

In [59]:
for model_type in precision_per_model_type_5_runs_3_classes:
    print(model_type)
    for class_name in precision_per_model_type_5_runs_3_classes[model_type]:
        print(class_name)
        precision = precision_per_model_type_5_runs_3_classes[model_type][class_name]['actual']/precision_per_model_type_5_runs_3_classes[model_type][class_name]['predicted']
        print(precision)
        precision_per_model_type_5_runs_3_classes[model_type][class_name]['precision'] = precision
        

Humans
Inconsistent
0.9122661122661123
Consistent
0.6305220883534136
Unrelated
0.5643086816720257
ChatGPT-4
Inconsistent
0.929495760821062
Unrelated
0.5376458112407211
Consistent
0.6083836858006042
ChatGPT-3.5
Inconsistent
0.7643969576240492
Consistent
0.7512755102040817
Unrelated
1.0
LLaMA 8B
Inconsistent
0.8003251702062798
Unrelated
0.2529500756429652
Consistent
0.2971645456575128
LLaMA 70B
Inconsistent
0.8773151578011558
Unrelated
0.7672583826429981
Consistent
0.6730621642363775


In [60]:
precision_per_model_type_5_runs_3_classes['bootstrap'] = dict()
for ground_truth, prediction in bootstrap_predictions_tuples_3_classes:
    try:
        precision_per_model_type_5_runs_3_classes['bootstrap'][prediction]['predicted'] +=1 
        if ground_truth == prediction:
            precision_per_model_type_5_runs_3_classes['bootstrap'][prediction]['actual'] +=1 
    except KeyError:
        precision_per_model_type_5_runs_3_classes['bootstrap'][prediction] = dict() 
        precision_per_model_type_5_runs_3_classes['bootstrap'][prediction]['predicted'] = 1
        precision_per_model_type_5_runs_3_classes['bootstrap'][prediction]['actual'] = 0
        if ground_truth == prediction:
            precision_per_model_type_5_runs_3_classes['bootstrap'][prediction]['actual'] +=1 

In [61]:
model_type = 'bootstrap'
for class_name in precision_per_model_type_5_runs_3_classes[model_type]:
    print(class_name)
    precision = precision_per_model_type_5_runs_3_classes[model_type][class_name]['actual']/precision_per_model_type_5_runs_3_classes[model_type][class_name]['predicted']
    print(precision)
    precision_per_model_type_5_runs_3_classes[model_type][class_name]['precision'] = precision

Inconsistent
0.9566205413301951
Consistent
0.7839206583667441
Unrelated
0.7348329925017042


In [62]:
# calculate total
for model_type in precision_per_model_type_5_runs_3_classes:
    total_actual = 0
    total_predicted = 0
    for class_name in precision_per_model_type_5_runs_3_classes[model_type]:
        total_actual += precision_per_model_type_5_runs_3_classes[model_type][class_name]['actual']
        total_predicted += precision_per_model_type_5_runs_3_classes[model_type][class_name]['predicted']
    total_precision = total_actual/total_predicted
    precision_per_model_type_5_runs_3_classes[model_type]['total'] = total_precision    

In [63]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\precision_per_model_type_5_runs_3_classes.pkl', 'wb') as f:
    pickle.dump(precision_per_model_type_5_runs_3_classes, f)

### Recalculate precision for 5 runs

In [31]:
precision_per_model_type_5_runs = dict()
for model_type in prediction_ground_truth_tuples_5_runs:
    precision_per_model_type_5_runs[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples_5_runs[model_type]:
        try:
            precision_per_model_type_5_runs[model_type][prediction]['predicted'] +=1 
            if ground_truth == prediction:
                precision_per_model_type_5_runs[model_type][prediction]['actual'] +=1 
        except KeyError:
            precision_per_model_type_5_runs[model_type][prediction] = dict() 
            precision_per_model_type_5_runs[model_type][prediction]['predicted'] = 1
            precision_per_model_type_5_runs[model_type][prediction]['actual'] = 0
            if ground_truth == prediction:
                precision_per_model_type_5_runs[model_type][prediction]['actual'] +=1 

In [32]:
for model_type in precision_per_model_type_5_runs:
    print(model_type)
    for class_name in precision_per_model_type_5_runs[model_type]:
        print(class_name)
        precision = precision_per_model_type_5_runs[model_type][class_name]['actual']/precision_per_model_type_5_runs[model_type][class_name]['predicted']
        print(precision)
        precision_per_model_type_5_runs[model_type][class_name]['precision'] = precision
        

Humans
Indirect inconsistency
0.45222222222222225
Factual inconsistency
0.38830584707646176
Consistent
0.6586345381526104
Surface contradiction
0.5632458233890215
Unrelated
0.5755627009646302
ChatGPT-4
Indirect inconsistency
0.3389049608914604
Unrelated
0.5620360551431601
Factual inconsistency
0.557919621749409
Surface contradiction
0.8118908382066277
Consistent
0.6423716012084593
ChatGPT-3.5
Factual inconsistency
0.2531270148291425
Surface contradiction
0.34036739031425534
Indirect inconsistency
0.37410071942446044
Consistent
0.7716836734693877
Unrelated
1.0
LLaMA 8B
Indirect inconsistency
0.2692307692307692
Factual inconsistency
0.22830930537352556
Surface contradiction
0.29064532266133064
Unrelated
0.2807866868381241
Consistent
0.3154722036168788
LLaMA 70B
Indirect inconsistency
0.3738140417457306
Unrelated
0.8001314924391848
Factual inconsistency
0.49659201557935734
Surface contradiction
0.6680227827502034
Consistent
0.6957022256331543


In [33]:
precision_per_model_type_5_runs['bootstrap'] = dict()
for ground_truth, prediction in bootstrap_predictions_tuples:
    try:
        precision_per_model_type_5_runs['bootstrap'][prediction]['predicted'] +=1 
        if ground_truth == prediction:
            precision_per_model_type_5_runs['bootstrap'][prediction]['actual'] +=1 
    except KeyError:
        precision_per_model_type_5_runs['bootstrap'][prediction] = dict() 
        precision_per_model_type_5_runs['bootstrap'][prediction]['predicted'] = 1
        precision_per_model_type_5_runs['bootstrap'][prediction]['actual'] = 0
        if ground_truth == prediction:
            precision_per_model_type_5_runs['bootstrap'][prediction]['actual'] +=1 

In [45]:
model_type = 'bootstrap'
for class_name in precision_per_model_type_5_runs[model_type]:
    print(class_name)
    precision = precision_per_model_type_5_runs[model_type][class_name]['actual']/precision_per_model_type_5_runs[model_type][class_name]['predicted']
    print(precision)
    precision_per_model_type_5_runs[model_type][class_name]['precision'] = precision

Indirect inconsistency
0.7039798267733801
Factual inconsistency
0.6408256880733945
Surface contradiction
0.7641819034224098
Consistent
0.7987704918032786
Unrelated
0.7657438886190707
total


TypeError: 'float' object is not subscriptable

In [35]:
# calculate total
for model_type in precision_per_model_type_5_runs:
    total_actual = 0
    total_predicted = 0
    for class_name in precision_per_model_type_5_runs[model_type]:
        total_actual += precision_per_model_type_5_runs[model_type][class_name]['actual']
        total_predicted += precision_per_model_type_5_runs[model_type][class_name]['predicted']
    total_precision = total_actual/total_predicted
    precision_per_model_type_5_runs[model_type]['total'] = total_precision    

In [47]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\precision_per_model_type_5_runs.pkl', 'wb') as f:
    pickle.dump(precision_per_model_type_5_runs, f)

### Recalculate precision for majority LLM - 3 classes

In [52]:
precision_per_model_type_majority_LLM_3_classes = dict()
for model_type in prediction_ground_truth_tuples_majority_LLM_3_classes:
    precision_per_model_type_majority_LLM_3_classes[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples_majority_LLM_3_classes[model_type]:
        try:
            precision_per_model_type_majority_LLM_3_classes[model_type][prediction]['predicted'] +=1 
            if ground_truth == prediction:
                precision_per_model_type_majority_LLM_3_classes[model_type][prediction]['actual'] +=1 
        except KeyError:
            precision_per_model_type_majority_LLM_3_classes[model_type][prediction] = dict() 
            precision_per_model_type_majority_LLM_3_classes[model_type][prediction]['predicted'] = 1
            precision_per_model_type_majority_LLM_3_classes[model_type][prediction]['actual'] = 0
            if ground_truth == prediction:
                precision_per_model_type_majority_LLM_3_classes[model_type][prediction]['actual'] +=1 

In [53]:
for model_type in precision_per_model_type_majority_LLM_3_classes:
    print(model_type)
    for class_name in precision_per_model_type_majority_LLM_3_classes[model_type]:
        print(class_name)
        precision = precision_per_model_type_majority_LLM_3_classes[model_type][class_name]['actual']/precision_per_model_type_majority_LLM_3_classes[model_type][class_name]['predicted']
        print(precision)
        precision_per_model_type_majority_LLM_3_classes[model_type][class_name]['precision'] = precision
        

Humans
Inconsistent
0.913929313929314
Consistent
0.6285140562248996
Unrelated
0.5466237942122186
ChatGPT-4
Inconsistent
0.9345211581291759
Unrelated
0.533068783068783
Consistent
0.6335877862595419
ChatGPT-3.5
Inconsistent
0.7690217391304348
Unrelated
1.0
Consistent
0.7777777777777778
LLaMA 8B
Inconsistent
0.7679197994987469
Consistent
0.2538293216630197
Unrelated
0.20292207792207792
LLaMA 70B
Inconsistent
0.8805750092148913
Unrelated
0.7722772277227723
Consistent
0.6777996070726916


In [54]:
precision_per_model_type_majority_LLM_3_classes['bootstrap'] = dict()
for ground_truth, prediction in bootstrap_predictions_tuples_3_classes:
    try:
        precision_per_model_type_majority_LLM_3_classes['bootstrap'][prediction]['predicted'] +=1 
        if ground_truth == prediction:
            precision_per_model_type_majority_LLM_3_classes['bootstrap'][prediction]['actual'] +=1 
    except KeyError:
        precision_per_model_type_majority_LLM_3_classes['bootstrap'][prediction] = dict() 
        precision_per_model_type_majority_LLM_3_classes['bootstrap'][prediction]['predicted'] = 1
        precision_per_model_type_majority_LLM_3_classes['bootstrap'][prediction]['actual'] = 0
        if ground_truth == prediction:
            precision_per_model_type_majority_LLM_3_classes['bootstrap'][prediction]['actual'] +=1 

In [55]:
# calculate total
for model_type in precision_per_model_type_majority_LLM_3_classes:
    total_actual = 0
    total_predicted = 0
    for class_name in precision_per_model_type_majority_LLM_3_classes[model_type]:
        total_actual += precision_per_model_type_majority_LLM_3_classes[model_type][class_name]['actual']
        total_predicted += precision_per_model_type_majority_LLM_3_classes[model_type][class_name]['predicted']
    total_precision = total_actual/total_predicted
    precision_per_model_type_majority_LLM_3_classes[model_type]['total'] = total_precision    

In [56]:
model_type = 'bootstrap'
for class_name in precision_per_model_type_majority_LLM_3_classes[model_type]:
    if class_name != 'total':
        print(class_name)
        precision = precision_per_model_type_majority_LLM_3_classes[model_type][class_name]['actual']/precision_per_model_type_majority_LLM_3_classes[model_type][class_name]['predicted']
        print(precision)
        precision_per_model_type_majority_LLM_3_classes[model_type][class_name]['precision'] = precision

Inconsistent
0.9566205413301951
Consistent
0.7839206583667441
Unrelated
0.7348329925017042


In [57]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\precision_per_model_type_majority_LLM_3_classes.pkl', 'wb') as f:
    pickle.dump(precision_per_model_type_majority_LLM_3_classes, f)

### Recalculate precision for majority LLM

In [25]:
precision_per_model_type_majority_LLM = dict()
for model_type in prediction_ground_truth_tuples_majority_LLM:
    precision_per_model_type_majority_LLM[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples_majority_LLM[model_type]:
        try:
            precision_per_model_type_majority_LLM[model_type][prediction]['predicted'] +=1 
            if ground_truth == prediction:
                precision_per_model_type_majority_LLM[model_type][prediction]['actual'] +=1 
        except KeyError:
            precision_per_model_type_majority_LLM[model_type][prediction] = dict() 
            precision_per_model_type_majority_LLM[model_type][prediction]['predicted'] = 1
            precision_per_model_type_majority_LLM[model_type][prediction]['actual'] = 0
            if ground_truth == prediction:
                precision_per_model_type_majority_LLM[model_type][prediction]['actual'] +=1 

In [26]:
for model_type in precision_per_model_type_majority_LLM:
    print(model_type)
    for class_name in precision_per_model_type_majority_LLM[model_type]:
        print(class_name)
        precision = precision_per_model_type_majority_LLM[model_type][class_name]['actual']/precision_per_model_type_majority_LLM[model_type][class_name]['predicted']
        print(precision)
        precision_per_model_type_majority_LLM[model_type][class_name]['precision'] = precision
        

Humans
Indirect inconsistency
0.45111111111111113
Factual inconsistency
0.3808095952023988
Consistent
0.6526104417670683
Surface contradiction
0.5715990453460621
Unrelated
0.6012861736334405
ChatGPT-4
Indirect inconsistency
0.33599149388623073
Unrelated
0.5793650793650794
Surface contradiction
0.8075117370892019
Consistent
0.6717557251908397
Factual inconsistency
0.5099337748344371
ChatGPT-3.5
Surface contradiction
0.35319767441860467
Factual inconsistency
0.2516469038208169
Indirect inconsistency
0.3708133971291866
Unrelated
1.0
Consistent
0.7908496732026143
LLaMA 8B
Indirect inconsistency
0.2361111111111111
Factual inconsistency
0.17657342657342656
Consistent
0.2735229759299781
Surface contradiction
0.24741298212605833
Unrelated
0.22077922077922077
LLaMA 70B
Indirect inconsistency
0.3694581280788177
Unrelated
0.8151815181518152
Surface contradiction
0.6744186046511628
Consistent
0.7033398821218074
Factual inconsistency
0.5047619047619047


In [27]:
precision_per_model_type_majority_LLM['bootstrap'] = dict()
for ground_truth, prediction in bootstrap_predictions_tuples:
    try:
        precision_per_model_type_majority_LLM['bootstrap'][prediction]['predicted'] +=1 
        if ground_truth == prediction:
            precision_per_model_type_majority_LLM['bootstrap'][prediction]['actual'] +=1 
    except KeyError:
        precision_per_model_type_majority_LLM['bootstrap'][prediction] = dict() 
        precision_per_model_type_majority_LLM['bootstrap'][prediction]['predicted'] = 1
        precision_per_model_type_majority_LLM['bootstrap'][prediction]['actual'] = 0
        if ground_truth == prediction:
            precision_per_model_type_majority_LLM['bootstrap'][prediction]['actual'] +=1 

In [28]:
# calculate total
for model_type in precision_per_model_type_majority_LLM:
    total_actual = 0
    total_predicted = 0
    for class_name in precision_per_model_type_majority_LLM[model_type]:
        total_actual += precision_per_model_type_majority_LLM[model_type][class_name]['actual']
        total_predicted += precision_per_model_type_majority_LLM[model_type][class_name]['predicted']
    total_precision = total_actual/total_predicted
    precision_per_model_type_majority_LLM[model_type]['total'] = total_precision    

In [48]:
model_type = 'bootstrap'
for class_name in precision_per_model_type_majority_LLM[model_type]:
    if class_name != 'total':
        print(class_name)
        precision = precision_per_model_type_majority_LLM[model_type][class_name]['actual']/precision_per_model_type_majority_LLM[model_type][class_name]['predicted']
        print(precision)
        precision_per_model_type_majority_LLM[model_type][class_name]['precision'] = precision

Indirect inconsistency
0.7039798267733801
Factual inconsistency
0.6408256880733945
Surface contradiction
0.7641819034224098
Consistent
0.7987704918032786
Unrelated
0.7657438886190707


In [50]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\precision_per_model_type_majority_LLM.pkl', 'wb') as f:
    pickle.dump(precision_per_model_type_majority_LLM, f)

### Recalculate recall for 5 runs LLM - 3 classes

In [40]:
recall_per_model_type_5_runs_3_classes = dict()


for model_type in prediction_ground_truth_tuples_5_runs_3_classes:
    recall_per_model_type_5_runs_3_classes[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples_5_runs_3_classes[model_type]:
        try:
            recall_per_model_type_5_runs_3_classes[model_type][ground_truth]['ground_truth_count'] +=1 
            if ground_truth == prediction:
                recall_per_model_type_5_runs_3_classes[model_type][prediction]['retrieved'] +=1 
        except KeyError:
            recall_per_model_type_5_runs_3_classes[model_type][ground_truth] = dict() 
            recall_per_model_type_5_runs_3_classes[model_type][ground_truth]['ground_truth_count'] = 1
            recall_per_model_type_5_runs_3_classes[model_type][ground_truth]['retrieved'] = 0
            if ground_truth == prediction:
                recall_per_model_type_5_runs_3_classes[model_type][prediction]['retrieved'] +=1 

In [41]:
for model_type in recall_per_model_type_5_runs_3_classes:
    print(model_type)
    for class_name in recall_per_model_type_5_runs_3_classes[model_type]:
        print(class_name)
        recall = recall_per_model_type_5_runs_3_classes[model_type][class_name]['retrieved']/recall_per_model_type_5_runs_3_classes[model_type][class_name]['ground_truth_count']
        print(recall)
        recall_per_model_type_5_runs_3_classes[model_type][class_name]['recall'] = recall
        

Humans
Inconsistent
0.8610675039246468
Unrelated
0.6370235934664247
Consistent
0.7370892018779343
ChatGPT-4
Inconsistent
0.8175039246467818
Unrelated
0.7361161524500908
Consistent
0.7563380281690141
ChatGPT-3.5
Inconsistent
0.993956043956044
Unrelated
0.0998185117967332
Consistent
0.2765258215962441
LLaMA 8B
Inconsistent
0.6182103610675039
Unrelated
0.30344827586206896
Consistent
0.6248826291079812
LLaMA 70B
Inconsistent
0.929513343799058
Unrelated
0.4235934664246824
Consistent
0.8234741784037559


In [42]:
model_type = 'bootstrap'
recall_per_model_type_5_runs_3_classes['bootstrap'] = dict()
            
for ground_truth, prediction in bootstrap_predictions_tuples_3_classes:
    try:
        recall_per_model_type_5_runs_3_classes[model_type][ground_truth]['ground_truth_count'] +=1 
        if ground_truth == prediction:
            recall_per_model_type_5_runs_3_classes[model_type][prediction]['retrieved'] +=1 
    except KeyError:
        recall_per_model_type_5_runs_3_classes[model_type][ground_truth] = dict() 
        recall_per_model_type_5_runs_3_classes[model_type][ground_truth]['ground_truth_count'] = 1
        recall_per_model_type_5_runs_3_classes[model_type][ground_truth]['retrieved'] = 0
        if ground_truth == prediction:
            recall_per_model_type_5_runs_3_classes[model_type][prediction]['retrieved'] +=1 

In [43]:
for class_name in recall_per_model_type_5_runs_3_classes['bootstrap']:
    recall = recall_per_model_type_5_runs_3_classes[model_type][class_name]['retrieved']/recall_per_model_type_5_runs_3_classes[model_type][class_name]['ground_truth_count']
    recall_per_model_type_5_runs_3_classes[model_type][class_name]['recall'] = recall

In [44]:
# calculate total
for model_type in recall_per_model_type_5_runs_3_classes:
    total_ground_truth = 0
    total_retrieved = 0
    for class_name in recall_per_model_type_5_runs_3_classes[model_type]:
        total_ground_truth += recall_per_model_type_5_runs_3_classes[model_type][class_name]['ground_truth_count']
        total_retrieved += recall_per_model_type_5_runs_3_classes[model_type][class_name]['retrieved']
    total_recall = total_retrieved/total_ground_truth
    recall_per_model_type_5_runs_3_classes[model_type]['total'] = total_recall    

In [45]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\recall_per_model_type_5_runs_3_classes.pkl', 'wb') as f:
    pickle.dump(recall_per_model_type_5_runs_3_classes, f)

### Recalculate recall for 5 runs LLM

In [19]:
recall_per_model_type_5_runs = dict()


for model_type in prediction_ground_truth_tuples_5_runs:
    recall_per_model_type_5_runs[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples_5_runs[model_type]:
        try:
            recall_per_model_type_5_runs[model_type][ground_truth]['ground_truth_count'] +=1 
            if ground_truth == prediction:
                recall_per_model_type_5_runs[model_type][prediction]['retrieved'] +=1 
        except KeyError:
            recall_per_model_type_5_runs[model_type][ground_truth] = dict() 
            recall_per_model_type_5_runs[model_type][ground_truth]['ground_truth_count'] = 1
            recall_per_model_type_5_runs[model_type][ground_truth]['retrieved'] = 0
            if ground_truth == prediction:
                recall_per_model_type_5_runs[model_type][prediction]['retrieved'] +=1 

In [20]:
for model_type in recall_per_model_type_5_runs:
    print(model_type)
    for class_name in recall_per_model_type_5_runs[model_type]:
        print(class_name)
        recall = recall_per_model_type_5_runs[model_type][class_name]['retrieved']/recall_per_model_type_5_runs[model_type][class_name]['ground_truth_count']
        print(recall)
        recall_per_model_type_5_runs[model_type][class_name]['recall'] = recall
        

Humans
Indirect inconsistency
0.4343649946638207
Surface contradiction
0.545664739884393
Unrelated
0.5830618892508144
Factual inconsistency
0.40723270440251574
Consistent
0.693446088794926
ChatGPT-4
Indirect inconsistency
0.6751334044823906
Surface contradiction
0.19260115606936415
Unrelated
0.6905537459283387
Factual inconsistency
0.14842767295597484
Consistent
0.7192389006342494
ChatGPT-3.5
Indirect inconsistency
0.1775880469583778
Surface contradiction
0.5183815028901734
Unrelated
0.08957654723127036
Factual inconsistency
0.6172955974842768
Consistent
0.2558139534883721
LLaMA 8B
Indirect inconsistency
0.11654215581643543
Surface contradiction
0.26867052023121385
Unrelated
0.3022801302931596
Factual inconsistency
0.2738993710691824
Consistent
0.5974630021141649
LLaMA 70B
Indirect inconsistency
0.7989327641408751
Surface contradiction
0.3796531791907514
Unrelated
0.3964169381107492
Factual inconsistency
0.16037735849056603
Consistent
0.7665961945031713


In [21]:
model_type = 'bootstrap'
recall_per_model_type_5_runs['bootstrap'] = dict()
            
for ground_truth, prediction in bootstrap_predictions_tuples:
    try:
        recall_per_model_type_5_runs[model_type][ground_truth]['ground_truth_count'] +=1 
        if ground_truth == prediction:
            recall_per_model_type_5_runs[model_type][prediction]['retrieved'] +=1 
    except KeyError:
        recall_per_model_type_5_runs[model_type][ground_truth] = dict() 
        recall_per_model_type_5_runs[model_type][ground_truth]['ground_truth_count'] = 1
        recall_per_model_type_5_runs[model_type][ground_truth]['retrieved'] = 0
        if ground_truth == prediction:
            recall_per_model_type_5_runs[model_type][prediction]['retrieved'] +=1 

In [22]:
for class_name in recall_per_model_type_5_runs['bootstrap']:
    recall = recall_per_model_type_5_runs[model_type][class_name]['retrieved']/recall_per_model_type_5_runs[model_type][class_name]['ground_truth_count']
    recall_per_model_type_5_runs[model_type][class_name]['recall'] = recall

In [23]:
# calculate total
for model_type in recall_per_model_type_5_runs:
    total_ground_truth = 0
    total_retrieved = 0
    for class_name in recall_per_model_type_5_runs[model_type]:
        total_ground_truth += recall_per_model_type_5_runs[model_type][class_name]['ground_truth_count']
        total_retrieved += recall_per_model_type_5_runs[model_type][class_name]['retrieved']
    total_recall = total_retrieved/total_ground_truth
    recall_per_model_type_5_runs[model_type]['total'] = total_recall    

In [24]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\recall_per_model_type_5_runs.pkl', 'wb') as f:
    pickle.dump(recall_per_model_type_5_runs, f)

### Recalculate recall for majority LLM - 3 classes

In [46]:
recall_per_model_type_majority_LLM_3_classes = dict()

for model_type in prediction_ground_truth_tuples_majority_LLM_3_classes:
    recall_per_model_type_majority_LLM_3_classes[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples_majority_LLM_3_classes[model_type]:
        try:
            recall_per_model_type_majority_LLM_3_classes[model_type][ground_truth]['ground_truth_count'] +=1 
            if ground_truth == prediction:
                recall_per_model_type_majority_LLM_3_classes[model_type][prediction]['retrieved'] +=1 
        except KeyError:
            recall_per_model_type_majority_LLM_3_classes[model_type][ground_truth] = dict() 
            recall_per_model_type_majority_LLM_3_classes[model_type][ground_truth]['ground_truth_count'] = 1
            recall_per_model_type_majority_LLM_3_classes[model_type][ground_truth]['retrieved'] = 0
            if ground_truth == prediction:
                recall_per_model_type_majority_LLM_3_classes[model_type][prediction]['retrieved'] +=1 

In [47]:
for model_type in recall_per_model_type_majority_LLM_3_classes:
    print(model_type)
    for class_name in recall_per_model_type_majority_LLM_3_classes[model_type]:
        print(class_name)
        recall = recall_per_model_type_majority_LLM_3_classes[model_type][class_name]['retrieved']/recall_per_model_type_majority_LLM_3_classes[model_type][class_name]['ground_truth_count']
        print(recall)
        recall_per_model_type_majority_LLM_3_classes[model_type][class_name]['recall'] = recall
        

Humans
Inconsistent
0.85859375
Unrelated
0.625
Consistent
0.7434679334916865
ChatGPT-4
Inconsistent
0.81953125
Unrelated
0.7408088235294118
Consistent
0.7885985748218527
ChatGPT-3.5
Inconsistent
0.994921875
Unrelated
0.11029411764705882
Consistent
0.2826603325415677
LLaMA 8B
Inconsistent
0.5984375
Unrelated
0.22977941176470587
Consistent
0.5510688836104513
LLaMA 70B
Inconsistent
0.933203125
Unrelated
0.43014705882352944
Consistent
0.8194774346793349


In [48]:
model_type = 'bootstrap'
recall_per_model_type_majority_LLM_3_classes['bootstrap'] = dict()
            
for ground_truth, prediction in bootstrap_predictions_tuples_3_classes:
    try:
        recall_per_model_type_majority_LLM_3_classes[model_type][ground_truth]['ground_truth_count'] +=1 
        if ground_truth == prediction:
            recall_per_model_type_majority_LLM_3_classes[model_type][prediction]['retrieved'] +=1 
    except KeyError:
        recall_per_model_type_majority_LLM_3_classes[model_type][ground_truth] = dict() 
        recall_per_model_type_majority_LLM_3_classes[model_type][ground_truth]['ground_truth_count'] = 1
        recall_per_model_type_majority_LLM_3_classes[model_type][ground_truth]['retrieved'] = 0
        if ground_truth == prediction:
            recall_per_model_type_majority_LLM_3_classes[model_type][prediction]['retrieved'] +=1 

In [49]:
for class_name in recall_per_model_type_majority_LLM_3_classes['bootstrap']:
    recall = recall_per_model_type_majority_LLM_3_classes[model_type][class_name]['retrieved']/recall_per_model_type_majority_LLM_3_classes[model_type][class_name]['ground_truth_count']
    recall_per_model_type_majority_LLM_3_classes[model_type][class_name]['recall'] = recall

In [50]:
# calculate total
for model_type in recall_per_model_type_majority_LLM_3_classes:
    total_ground_truth = 0
    total_retrieved = 0
    for class_name in recall_per_model_type_majority_LLM_3_classes[model_type]:
        total_ground_truth += recall_per_model_type_majority_LLM_3_classes[model_type][class_name]['ground_truth_count']
        total_retrieved += recall_per_model_type_majority_LLM_3_classes[model_type][class_name]['retrieved']
    total_recall = total_retrieved/total_ground_truth
    recall_per_model_type_majority_LLM_3_classes[model_type]['total'] = total_recall    

In [51]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\recall_per_model_type_majority_LLM_3_classes.pkl', 'wb') as f:
    pickle.dump(recall_per_model_type_majority_LLM_3_classes, f)

### Recalculate recall for majority LLM

In [13]:
recall_per_model_type_majority_LLM = dict()

for model_type in prediction_ground_truth_tuples_majority_LLM:
    recall_per_model_type_majority_LLM[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples_majority_LLM[model_type]:
        try:
            recall_per_model_type_majority_LLM[model_type][ground_truth]['ground_truth_count'] +=1 
            if ground_truth == prediction:
                recall_per_model_type_majority_LLM[model_type][prediction]['retrieved'] +=1 
        except KeyError:
            recall_per_model_type_majority_LLM[model_type][ground_truth] = dict() 
            recall_per_model_type_majority_LLM[model_type][ground_truth]['ground_truth_count'] = 1
            recall_per_model_type_majority_LLM[model_type][ground_truth]['retrieved'] = 0
            if ground_truth == prediction:
                recall_per_model_type_majority_LLM[model_type][prediction]['retrieved'] +=1 

In [14]:
for model_type in recall_per_model_type_majority_LLM:
    print(model_type)
    for class_name in recall_per_model_type_majority_LLM[model_type]:
        print(class_name)
        recall = recall_per_model_type_majority_LLM[model_type][class_name]['retrieved']/recall_per_model_type_majority_LLM[model_type][class_name]['ground_truth_count']
        print(recall)
        recall_per_model_type_majority_LLM[model_type][class_name]['recall'] = recall
        

Humans
Indirect inconsistency
0.43609022556390975
Surface contradiction
0.5468036529680366
Unrelated
0.5955414012738853
Factual inconsistency
0.40445859872611467
Consistent
0.7034632034632035
ChatGPT-4
Indirect inconsistency
0.6788399570354458
Surface contradiction
0.1963470319634703
Unrelated
0.697452229299363
Factual inconsistency
0.12261146496815287
Consistent
0.7619047619047619
ChatGPT-3.5
Indirect inconsistency
0.16648764769065522
Surface contradiction
0.5547945205479452
Unrelated
0.09554140127388536
Factual inconsistency
0.60828025477707
Consistent
0.2619047619047619
LLaMA 8B
Indirect inconsistency
0.09129967776584318
Surface contradiction
0.3002283105022831
Unrelated
0.21656050955414013
Factual inconsistency
0.160828025477707
Consistent
0.5411255411255411
LLaMA 70B
Indirect inconsistency
0.8055853920515574
Surface contradiction
0.3641552511415525
Unrelated
0.39331210191082805
Factual inconsistency
0.16878980891719744
Consistent
0.7748917748917749


In [15]:
model_type = 'bootstrap'
recall_per_model_type_majority_LLM['bootstrap'] = dict()
            
for ground_truth, prediction in bootstrap_predictions_tuples:
    try:
        recall_per_model_type_majority_LLM[model_type][ground_truth]['ground_truth_count'] +=1 
        if ground_truth == prediction:
            recall_per_model_type_majority_LLM[model_type][prediction]['retrieved'] +=1 
    except KeyError:
        recall_per_model_type_majority_LLM[model_type][ground_truth] = dict() 
        recall_per_model_type_majority_LLM[model_type][ground_truth]['ground_truth_count'] = 1
        recall_per_model_type_majority_LLM[model_type][ground_truth]['retrieved'] = 0
        if ground_truth == prediction:
            recall_per_model_type_majority_LLM[model_type][prediction]['retrieved'] +=1 

In [16]:
for class_name in recall_per_model_type_majority_LLM['bootstrap']:
    recall = recall_per_model_type_majority_LLM[model_type][class_name]['retrieved']/recall_per_model_type_majority_LLM[model_type][class_name]['ground_truth_count']
    recall_per_model_type_majority_LLM[model_type][class_name]['recall'] = recall

In [17]:
# calculate total
for model_type in recall_per_model_type_majority_LLM:
    total_ground_truth = 0
    total_retrieved = 0
    for class_name in recall_per_model_type_majority_LLM[model_type]:
        total_ground_truth += recall_per_model_type_majority_LLM[model_type][class_name]['ground_truth_count']
        total_retrieved += recall_per_model_type_majority_LLM[model_type][class_name]['retrieved']
    total_recall = total_retrieved/total_ground_truth
    recall_per_model_type_majority_LLM[model_type]['total'] = total_recall    

In [18]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\recall_per_model_type_majority_LLM.pkl', 'wb') as f:
    pickle.dump(recall_per_model_type_majority_LLM, f)

### Recalculate cross-validation for majority

In [10]:
prediction_ground_truth_tuples_majority_LLM = dict()
prediction_ground_truth_tuples_majority_LLM['Humans'] = []
prediction_ground_truth_tuples_majority_LLM['ChatGPT-4'] = []
prediction_ground_truth_tuples_majority_LLM['ChatGPT-3.5'] = []
prediction_ground_truth_tuples_majority_LLM['LLaMA 8B'] = []
prediction_ground_truth_tuples_majority_LLM['LLaMA 70B'] = []

In [11]:
for i in range(len(df_gold)):
    # we have 5 tuples per sample 
    # ground truth - from majority, the 5th one is prediction 
    strings = eval(df_gold['All answers'][i])
    # create N tuples of size (N-1), 1 held-out
    versions = [(strings[j], strings[:j] + strings[j+1:]) for j in range(len(strings))]
    # go through each element
    # value is held out "prediction"
    for value, version in versions:
        list_annotator_dropped = version
        counter = Counter(list_annotator_dropped)
        # identify the majority vote and sample randomly if more than 1
        max_count = max(counter.values())
        top_labels = [label for label, count in counter.items() if count == max_count]
        majority = random.choice(top_labels)
        prediction_ground_truth_tuples_majority_LLM['Humans'].append((majority, value)) #ground truth, prediction
        prediction_ground_truth_tuples_majority_LLM['ChatGPT-4'].append((majority, df_gold['ChatGPT-4 majority'][i]))
        prediction_ground_truth_tuples_majority_LLM['ChatGPT-3.5'].append((majority, df_gold['ChatGPT-3.5 majority'][i]))
        prediction_ground_truth_tuples_majority_LLM['LLaMA 8B'].append((majority, df_gold['Llama 8B majority'][i]))
        prediction_ground_truth_tuples_majority_LLM['LLaMA 70B'].append((majority, df_gold['Llama 70B majority'][i]))
    
        
    

In [12]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\prediction_ground_truth_tuples_majority_LLM.json', 'w') as f:
    json.dump(prediction_ground_truth_tuples_majority_LLM, f)

### Recalculate cross-validation for majority - 3 classes

In [7]:
prediction_ground_truth_tuples_majority_LLM_3_classes = dict()
prediction_ground_truth_tuples_majority_LLM_3_classes['Humans'] = []
prediction_ground_truth_tuples_majority_LLM_3_classes['ChatGPT-4'] = []
prediction_ground_truth_tuples_majority_LLM_3_classes['ChatGPT-3.5'] = []
prediction_ground_truth_tuples_majority_LLM_3_classes['LLaMA 8B'] = []
prediction_ground_truth_tuples_majority_LLM_3_classes['LLaMA 70B'] = []

In [8]:
for i in range(len(df_gold)):
    # we have 5 tuples per sample 
    # ground truth - from majority, the 5th one is prediction 
    strings = eval(df_gold['All answers'][i])
    # create N tuples of size (N-1), 1 held-out
    versions = [(strings[j], strings[:j] + strings[j+1:]) for j in range(len(strings))]
    # go through each element
    # value is held out "prediction"
    for value, version in versions:
        ####
        list_annotator_dropped = [classes_5_to_3[el] for el in version]
        ####
        value = classes_5_to_3[value]
        counter = Counter(list_annotator_dropped)
        # identify the majority vote and sample randomly if more than 1
        max_count = max(counter.values())
        top_labels = [label for label, count in counter.items() if count == max_count]
        majority = random.choice(top_labels)
        prediction_ground_truth_tuples_majority_LLM_3_classes['Humans'].append((majority, value)) #ground truth, prediction
        prediction_ground_truth_tuples_majority_LLM_3_classes['ChatGPT-4'].append((majority, classes_5_to_3[df_gold['ChatGPT-4 majority'][i]]))
        prediction_ground_truth_tuples_majority_LLM_3_classes['ChatGPT-3.5'].append((majority, classes_5_to_3[df_gold['ChatGPT-3.5 majority'][i]]))
        prediction_ground_truth_tuples_majority_LLM_3_classes['LLaMA 8B'].append((majority, classes_5_to_3[df_gold['Llama 8B majority'][i]]))
        prediction_ground_truth_tuples_majority_LLM_3_classes['LLaMA 70B'].append((majority, classes_5_to_3[df_gold['Llama 70B majority'][i]]))
    
        
    

In [9]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\prediction_ground_truth_tuples_majority_LLM_3_classes.json', 'w') as f:
    json.dump(prediction_ground_truth_tuples_majority_LLM_3_classes, f)

### Recalculate cross-validation for all 5 runs - 3 classes

In [5]:
prediction_ground_truth_tuples_5_runs_3_classes = dict()
prediction_ground_truth_tuples_5_runs_3_classes['Humans'] = []
prediction_ground_truth_tuples_5_runs_3_classes['ChatGPT-4'] = []
prediction_ground_truth_tuples_5_runs_3_classes['ChatGPT-3.5'] = []
prediction_ground_truth_tuples_5_runs_3_classes['LLaMA 8B'] = []
prediction_ground_truth_tuples_5_runs_3_classes['LLaMA 70B'] = []

In [6]:
for i in range(len(df_gold)):
    # we have 5 tuples per sample 
    # ground truth - from majority, the 5th one is prediction 
    strings = eval(df_gold['All answers'][i])
    # create N tuples of size (N-1), 1 held-out
    versions = [(strings[j], strings[:j] + strings[j+1:]) for j in range(len(strings))]
    # go through each element
    # value is held out "prediction"
    for value, version in versions:
        ####
        list_annotator_dropped = [classes_5_to_3[el] for el in version]
        counter = Counter(list_annotator_dropped)
        # identify the majority vote and sample randomly if more than 1
        max_count = max(counter.values())
        top_labels = [label for label, count in counter.items() if count == max_count]
        majority = random.choice(top_labels)
        ####
        value = classes_5_to_3[value]
        prediction_ground_truth_tuples_5_runs_3_classes['Humans'].append((majority, value)) #ground truth, prediction

        # compare each majority with each of the 5 runs results of the model 
        for run in range(5):
            prediction_ground_truth_tuples_5_runs_3_classes['ChatGPT-4'].append((majority, classes_5_to_3[eval(df_gold['ChatGPT-4 5 runs'][i])[run]]))
            prediction_ground_truth_tuples_5_runs_3_classes['ChatGPT-3.5'].append((majority, classes_5_to_3[eval(df_gold['ChatGPT-3.5 5 runs'][i])[run]]))
            prediction_ground_truth_tuples_5_runs_3_classes['LLaMA 8B'].append((majority, classes_5_to_3[eval(df_gold['LLaMA 8B 5 runs'][i])[run]]))
            prediction_ground_truth_tuples_5_runs_3_classes['LLaMA 70B'].append((majority, classes_5_to_3[eval(df_gold['LLaMA 70B 5 runs'][i])[run]]))
        
        
    

In [10]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\prediction_ground_truth_tuples_5_runs_3_classes.json', 'w') as f:
    json.dump(prediction_ground_truth_tuples_5_runs_3_classes, f)

### Recalculate cross-validation for all 5 runs

In [4]:
prediction_ground_truth_tuples_5_runs = dict()
prediction_ground_truth_tuples_5_runs['Humans'] = []
prediction_ground_truth_tuples_5_runs['ChatGPT-4'] = []
prediction_ground_truth_tuples_5_runs['ChatGPT-3.5'] = []
prediction_ground_truth_tuples_5_runs['LLaMA 8B'] = []
prediction_ground_truth_tuples_5_runs['LLaMA 70B'] = []

In [6]:
for i in range(len(df_gold)):
    # we have 5 tuples per sample 
    # ground truth - from majority, the 5th one is prediction 
    strings = eval(df_gold['All answers'][i])
    # create N tuples of size (N-1), 1 held-out
    versions = [(strings[j], strings[:j] + strings[j+1:]) for j in range(len(strings))]
    # go through each element
    # value is held out "prediction"
    for value, version in versions:
        list_annotator_dropped = version
        counter = Counter(list_annotator_dropped)
        # identify the majority vote and sample randomly if more than 1
        max_count = max(counter.values())
        top_labels = [label for label, count in counter.items() if count == max_count]
        majority = random.choice(top_labels)
        prediction_ground_truth_tuples_5_runs['Humans'].append((majority, value)) #ground truth, prediction

        # compare each majority with each of the 5 runs results of the model 
        for run in range(5):
            prediction_ground_truth_tuples_5_runs['ChatGPT-4'].append((majority, eval(df_gold['ChatGPT-4 5 runs'][i])[run]))
            prediction_ground_truth_tuples_5_runs['ChatGPT-3.5'].append((majority, eval(df_gold['ChatGPT-3.5 5 runs'][i])[run]))
            prediction_ground_truth_tuples_5_runs['LLaMA 8B'].append((majority, eval(df_gold['LLaMA 8B 5 runs'][i])[run]))
            prediction_ground_truth_tuples_5_runs['LLaMA 70B'].append((majority, eval(df_gold['LLaMA 70B 5 runs'][i])[run]))
        
        
    

In [8]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\prediction_ground_truth_tuples_5_runs.json', 'w') as f:
    json.dump(prediction_ground_truth_tuples_5_runs, f)

In [9]:
len(prediction_ground_truth_tuples_5_runs['Humans'])

3525

In [40]:
17625/3525

5.0

In [57]:
(5 * 660 * 5) + (6 * 30 * 5) + (7 * 8 * 5)

17680

### Recalculate precision - cross-validation for all 5 runs

### Recalculating agreement - but now we concatenate all tuples for a class

Now total F1 macro score

### Evaluate LLaMa 70B

#### 5 classes

In [4]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_3_70B_instruct_5_types.pkl", 'rb') as f:
    llama_70B_run_1 = pickle.load(f)

In [5]:
### replace "\n" with "\n\n"
key_changes = dict()

for el in llama_70B_run_1.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in llama_70B_run_1:
        llama_70B_run_1[new_key] = llama_70B_run_1.pop(old_key)

In [6]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_llama3_3_70B_instruct_5_types_run_2.pkl", 'rb') as f:
    llama_70B_run_2 = pickle.load(f)
    
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_llama3_3_70B_instruct_5_types_run_3.pkl", 'rb') as f:
    llama_70B_run_3 = pickle.load(f)
    
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_llama3_3_70B_instruct_5_types_run_4.pkl", 'rb') as f:
    llama_70B_run_4 = pickle.load(f)
    
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_llama3_3_70B_instruct_5_types_run_5.pkl", 'rb') as f:
    llama_70B_run_5 = pickle.load(f)

In [19]:
llama_70B_sample_to_runs = dict()
for i in range(len(df_gold)):
    sample = df_gold['Input'][i]
    llama_70B_sample_to_runs[sample] = []
    llama_70B_sample_to_runs[sample].append(df_gold['LLaMA 70B 5 classes'][i])
    for run_N in [llama_70B_run_2, llama_70B_run_3, llama_70B_run_4, llama_70B_run_5]:
        res = run_N[sample].content
        found_classes = [cls_name for cls_name in classes_5_casefold if cls_name in res]
        if len(found_classes) > 1:
            print(found_classes)
            print(res)
        pred = found_classes[0]
        pred = pred.lower().capitalize()
        llama_70B_sample_to_runs[sample].append(pred)


In [25]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\llama_70B_sample_to_runs.json', 'w') as f:
    json.dump(llama_70B_sample_to_runs, f)

In [62]:
wrong_to_correct_label_3_types = {"Indirect inconsistency": "Inconsistent", "Indirect Inconsistency": "Inconsistent",
                                  "Factual inconsistency": "Inconsistent", "Factual Inconsistency": "Inconsistent",
                                  "Surface contradiction": "Inconsistent", "Surface Contradiction": "Inconsistent", 
                                'Unrelated': 'Unrelated', 'Consistent': 'Consistent',
                                   'Inconsistent':  'Inconsistent'} 

### Evaluate LLaMa 8B

#### 5 classes

In [27]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_8B_instruct_5_types.pkl", 'rb') as f:
    llama_8B_run_1 = pickle.load(f)

In [58]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_llama3_8B_instruct_5_types_run_2.pkl", 'rb') as f:
    llama_8B_run_2 = pickle.load(f)
    
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_llama3_8B_instruct_5_types_run_3.pkl", 'rb') as f:
    llama_8B_run_3 = pickle.load(f)
    
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_llama3_8B_instruct_5_types_run_4.pkl", 'rb') as f:
    llama_8B_run_4 = pickle.load(f)
    
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_llama3_8B_instruct_5_types_run_5.pkl", 'rb') as f:
    llama_8B_run_5 = pickle.load(f)

In [30]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in llama_8B_run_1.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in llama_8B_run_1:
        llama_8B_run_1[new_key] = llama_8B_run_1.pop(old_key)

In [59]:
samples_to_rerun = []
llama_8B_sample_to_runs = dict()
for i in range(len(df_gold)):
    sample = df_gold['Input'][i]
    llama_8B_sample_to_runs[sample] = []
    llama_8B_sample_to_runs[sample].append(df_gold['LLaMA 5 classes'][i])
    for run_N in range(4):
        model_run = [llama_8B_run_2, llama_8B_run_3, llama_8B_run_4, llama_8B_run_5][run_N]
        res = model_run[sample].content
        found_classes = [cls_name for cls_name in classes_5_casefold if cls_name in res]
        if len(found_classes) > 1:
            print(run_N+2)
            print(found_classes)
            print(res)
            samples_to_rerun.append(sample)
        try:
            pred = found_classes[0]
        except:
            if "Explanation: Text 1 suggests separating disabled and non-disabled individuals in schools, while Text 2 advocates for the inclusion of all students with disabilities in regular schools. The two statements go in opposite directions, showing a surface-level contradiction." in res:
                pred = 'Surface contradiction'
        pred = pred.lower().capitalize()
        llama_8B_sample_to_runs[sample].append(pred)


In [62]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\llama_8B_sample_to_runs.json", 'w') as f:
    json.dump(llama_8B_sample_to_runs, f)

In [5]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\llama_8B_sample_to_runs.json", 'r') as f:
    llama_8B_sample_to_runs = json.load(f)

In [7]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\llama_70B_sample_to_runs.json", 'r') as f:
    llama_70B_sample_to_runs = json.load(f)

In [6]:
df_gold['LLaMA 8B 5 runs'] = 0
for i in range(len(df_gold)):
    sample = df_gold['Input'][i]
    df_gold['LLaMA 8B 5 runs'][i] = llama_8B_sample_to_runs[sample]

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_205992\975845271.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['LLaMA 8B 5 runs'][i] = llama_8B_sample_to_runs[sample]
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_205992\975845271.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Indirect inconsistency', 'Factual inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['LLaMA 8B 5 runs'][i] = llama_8B_sample_to_runs[sample]


In [52]:
for i in range(len(df_gold)):
    print(df_gold['LLaMA 8B 5 runs'][i])

['Indirect inconsistency', 'Factual inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency']
['Factual inconsistency', 'Surface contradiction', 'Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency']
['Factual inconsistency', 'Indirect inconsistency', 'Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency']
['Unrelated', 'Consistent', 'Consistent', 'Consistent', 'Consistent']
['Unrelated', 'Consistent', 'Consistent', 'Consistent', 'Consistent']
['Surface contradiction', 'Indirect inconsistency', 'Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency']
['Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency']
['Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency', 'Factual inconsistency']
['Indirect inconsistency', 'Unrelated', 'Surface contradiction', 'Surface contradiction', 'Surface

In [8]:
df_gold['LLaMA 70B 5 runs'] = 0
for i in range(len(df_gold)):
    sample = df_gold['Input'][i]
    df_gold['LLaMA 70B 5 runs'][i] = llama_70B_sample_to_runs[sample]

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_205992\3940489300.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['LLaMA 70B 5 runs'][i] = llama_70B_sample_to_runs[sample]
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_205992\3940489300.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['LLaMA 70B 5 runs'][i] = llama_70B_sample_to_runs[sample]


In [14]:
counter_switch = 0
num_switches = []
for i in range(len(df_gold)):
    answers = df_gold['LLaMA 8B 5 runs'][i]
    if len(set(answers)) > 1:
        counter_switch += 1
        num_switches.append(len(set(answers)))

In [15]:
Counter(num_switches)

Counter({2: 387, 3: 156})

In [16]:
counter_switch = 0
num_switches = []
for i in range(len(df_gold)):
    answers = df_gold['LLaMA 70B 5 runs'][i]
    if len(set(answers)) > 1:
        counter_switch += 1
        num_switches.append(len(set(answers)))

In [17]:
Counter(num_switches)

Counter({2: 116, 3: 7})

In [13]:
counter_switch

543

In [31]:
df_gold.to_csv(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv', index=False)

### Eval ChatGPT-3.5 turbo, 5 types

In [6]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt35_turbo_5_types.pkl', 'rb') as f:
    chatgpt35_run_1 = pickle.load(f)

In [7]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in chatgpt35_run_1.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in chatgpt35_run_1:
        chatgpt35_run_1[new_key] = chatgpt35_run_1.pop(old_key)

In [8]:
len(chatgpt35_run_1)

698

In [10]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt35_turbo_5_types_run2.pkl', 'rb') as f:
    chatgpt35_run_2 = pickle.load(f)
    
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt35_turbo_5_types_run3.pkl', 'rb') as f:
    chatgpt35_run_3 = pickle.load(f)
    
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt35_turbo_5_types_run4.pkl', 'rb') as f:
    chatgpt35_run_4 = pickle.load(f)
    
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt35_turbo_5_types_run5.pkl', 'rb') as f:
    chatgpt35_run_5 = pickle.load(f)

In [118]:
df_gold['ChatGPT-3.5 5 runs'] = 0

Change here 2 samples --> rerun everything else.

In [11]:
samples_to_rerun = []

chatgpt35_sample_to_runs = dict()
for i in range(len(df_gold)):
    sample = df_gold['Input'][i]
    chatgpt35_sample_to_runs[sample] = []
    chatgpt35_sample_to_runs[sample].append(df_gold['ChatGPT-3.5 5 classes'][i])
    for j in range(4):
        run_N = [chatgpt35_run_2, chatgpt35_run_3, chatgpt35_run_4, chatgpt35_run_5][j]
        pred = run_N[sample]['label']
        if pred not in classes_5_casefold:
            print(j+2)
            samples_to_rerun.append(sample)
            print(sample)
            print(pred)
            print(run_N[sample])
        chatgpt35_sample_to_runs[sample].append(pred)

In [13]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\chatgpt35_sample_to_runs.json', 'w') as f:
    json.dump(chatgpt35_sample_to_runs, f)

In [14]:
df_gold['ChatGPT-3.5 5 runs'] = 0
for i in range(len(df_gold)):
    sample = df_gold['Input'][i]
    df_gold['ChatGPT-3.5 5 runs'][i] = chatgpt35_sample_to_runs[sample]

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_231616\1254092444.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['ChatGPT-3.5 5 runs'][i] = chatgpt35_sample_to_runs[sample]
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_231616\1254092444.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Factual inconsistency', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction', 'Surface contradiction']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['ChatGPT-3.5 5 runs'][i] = chatgpt35_sample_to_runs[sample]


### Eval ChatGPT-4 turbo, 5 types

In [19]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt4_turbo_5_types_run1.pkl', 'rb') as f:
    chatgpt4_run_1 = pickle.load(f)

with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt4_turbo_5_types_run2.pkl', 'rb') as f:
    chatgpt4_run_2 = pickle.load(f)
    
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt4_turbo_5_types_run3.pkl', 'rb') as f:
    chatgpt4_run_3 = pickle.load(f)
    
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt4_turbo_5_types_run4.pkl', 'rb') as f:
    chatgpt4_run_4 = pickle.load(f)
    
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\new_run\answers_chatgpt4_turbo_5_types_run5.pkl', 'rb') as f:
    chatgpt4_run_5 = pickle.load(f)

In [20]:
statements_to_rerun = []
chatgpt4_sample_to_runs = dict()
for i in range(len(df_gold)):
    sample = df_gold['Input'][i]
    chatgpt4_sample_to_runs[sample] = []
    for j in range(5):
        run_N = [chatgpt4_run_1, chatgpt4_run_2, chatgpt4_run_3, chatgpt4_run_4, chatgpt4_run_5][j]
        pred = run_N[sample]['label']
        if pred not in classes_5_casefold:
            print(j+1)
            print(sample)
            print(pred)
            print(run_N[sample])
            statements_to_rerun.append(sample)
            
        chatgpt4_sample_to_runs[sample].append(pred)

In [21]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\chatgpt4_sample_to_runs.json', 'w') as f:
    json.dump(chatgpt4_sample_to_runs, f)

In [22]:
df_gold['ChatGPT-4 5 runs'] = 0
for i in range(len(df_gold)):
    sample = df_gold['Input'][i]
    df_gold['ChatGPT-4 5 runs'][i] = chatgpt4_sample_to_runs[sample]

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_230140\4255906400.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['ChatGPT-4 5 runs'][i] = chatgpt4_sample_to_runs[sample]
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_230140\4255906400.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency', 'Indirect inconsistency']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['ChatGPT-4 5 runs'][i] = chatgpt4_sample_to_runs[sample]


### Identify majority (3 classes)

In [9]:
df_gold['Llama 8B majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['LLaMA 8B 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['Llama 8B majority 3 class'][i] = majority
    
    

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_253480\3104221704.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Llama 8B majority 3 class'][i] = majority
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_253480\3104221704.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['Llama 8B majority 3 class'][i] = majority


In [11]:
df_gold['Llama 70B majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['LLaMA 70B 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['Llama 70B majority 3 class'][i] = majority
    
    

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_253480\1261665021.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Llama 70B majority 3 class'][i] = majority
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_253480\1261665021.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['Llama 70B majority 3 class'][i] = majority


In [12]:
df_gold['ChatGPT-3.5 majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['ChatGPT-3.5 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-3.5 majority 3 class'][i] = majority

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_253480\2368924386.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['ChatGPT-3.5 majority 3 class'][i] = majority
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_253480\2368924386.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['ChatGPT-3.5 majority 3 class'][i] = majority


In [13]:
df_gold['ChatGPT-4 majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['ChatGPT-4 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-4 majority 3 class'][i] = majority


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_253480\2873034803.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['ChatGPT-4 majority 3 class'][i] = majority
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_253480\2873034803.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['ChatGPT-4 majority 3 class'][i] = majority


### Identify majority

In [26]:
df_gold['Llama 8B majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['LLaMA 8B 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['Llama 8B majority'][i] = majority
    
    

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_205992\2759030784.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Llama 8B majority'][i] = majority
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_205992\2759030784.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Indirect inconsistency' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['Llama 8B majority'][i] = majority


In [40]:
df_gold['Llama 70B majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['LLaMA 70B 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['Llama 70B majority'][i] = majority
    
    

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_205992\3831204884.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Llama 70B majority'][i] = majority
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_205992\3831204884.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Indirect inconsistency' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['Llama 70B majority'][i] = majority


In [27]:
df_gold_diff = df_gold[df_gold['LLaMA 5 classes'] != df_gold['Llama 8B majority']]

In [41]:
df_gold_diff = df_gold[df_gold['LLaMA 70B 5 classes'] != df_gold['Llama 70B majority']]

In [16]:
df_gold['ChatGPT-3.5 majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['ChatGPT-3.5 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-3.5 majority'][i] = majority


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_231616\4009828070.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['ChatGPT-3.5 majority'][i] = majority
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_231616\4009828070.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Surface contradiction' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['ChatGPT-3.5 majority'][i] = majority


In [23]:
df_gold['ChatGPT-4 majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['ChatGPT-4 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-4 majority'][i] = majority


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_230140\4261485618.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['ChatGPT-4 majority'][i] = majority
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_230140\4261485618.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Indirect inconsistency' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold['ChatGPT-4 majority'][i] = majority


In [30]:
df_gold_diff = df_gold[df_gold['ChatGPT-4 5 classes'] != df_gold['ChatGPT-4 majority']]

In [31]:
df_gold_diff

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,Model_agreement_with_human,model_agreement_with_human_3_classes,ChatGPT-3.5 5 classes,...,NLI_ChatGPT4_turbo,Avg_synthetic_agreement,LLaMA 8B 5 runs,LLaMA 70B 5 runs,Llama 8B majority,Llama 70B majority,ChatGPT-3.5 5 runs,ChatGPT-4 5 runs,ChatGPT-3.5 majority,ChatGPT-4 majority
3,Text 1: We support the implementation of pilot...,Unrelated,60.0,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,Indirect inconsistency,Unrelated,0.2,0.6,Indirect inconsistency,...,Unrelated,0.7,"['Unrelated', 'Consistent', 'Consistent', 'Con...","['Indirect inconsistency', 'Indirect inconsist...",Consistent,Indirect inconsistency,"[Indirect inconsistency, Indirect inconsistenc...","[Unrelated, Unrelated, Unrelated, Unrelated, U...",Indirect inconsistency,Unrelated
5,Text 1: In the unresolved issue of waste stora...,Indirect inconsistency,40.0,"['Surface contradiction', 'Indirect inconsiste...",Inconsistent,Factual inconsistency,Inconsistent,0.2,0.8,Factual inconsistency,...,Unrelated,0.5,"['Surface contradiction', 'Indirect inconsiste...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,"[Factual inconsistency, Factual inconsistency,...","[Indirect inconsistency, Indirect inconsistenc...",Factual inconsistency,Indirect inconsistency
6,Text 1: We should not promote any more project...,Unrelated,40.0,"['Unrelated', 'Surface contradiction', 'Factua...",Unrelated,Surface contradiction,Inconsistent,0.2,0.6,Factual inconsistency,...,Unrelated,0.5,"['Factual inconsistency', 'Factual inconsisten...","['Unrelated', 'Indirect inconsistency', 'Indir...",Factual inconsistency,Indirect inconsistency,"[Factual inconsistency, Surface contradiction,...","[Indirect inconsistency, Indirect inconsistenc...",Surface contradiction,Indirect inconsistency
17,"Text 1: All people have a right to a homeland,...",Surface contradiction,80.0,"['Surface contradiction', 'Surface contradicti...",Inconsistent,Factual inconsistency,Inconsistent,0.2,1.0,Factual inconsistency,...,Contradiction,1.0,"['Factual inconsistency', 'Factual inconsisten...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,"[Factual inconsistency, Factual inconsistency,...","[Indirect inconsistency, Indirect inconsistenc...",Factual inconsistency,Indirect inconsistency
19,Text 1: Europol is less controllable than nati...,Indirect inconsistency,60.0,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Unrelated,0.0,0.0,Surface contradiction,...,Unrelated,1.0,"['Factual inconsistency', 'Factual inconsisten...","['Indirect inconsistency', 'Indirect inconsist...",Factual inconsistency,Indirect inconsistency,"[Surface contradiction, Surface contradiction,...","[Unrelated, Unrelated, Unrelated, Unrelated, U...",Surface contradiction,Unrelated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,Text 1: The free decision-making power of busi...,Surface contradiction,60.0,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,Surface contradiction,Inconsistent,0.6,1.0,Surface contradiction,...,Unrelated,0.7,"['Factual inconsistency', 'Factual inconsisten...","['Surface contradiction', 'Surface contradicti...",Consistent,Surface contradiction,"[Surface contradiction, Surface contradiction,...","[Indirect inconsistency, Indirect inconsistenc...",Surface contradiction,Indirect inconsistency
685,Text 1: We support lowering the pension conver...,Factual inconsistency,60.0,"['Unrelated', 'Factual inconsistency', 'Factua...",Inconsistent,Factual inconsistency,Inconsistent,0.6,0.8,Factual inconsistency,...,Contradiction,0.7,"['Consistent', 'Factual inconsistency', 'Factu...","['Factual inconsistency', 'Factual inconsisten...",Factual inconsistency,Factual inconsistency,"[Factual inconsistency, Factual inconsistency,...","[Indirect inconsistency, 

In [32]:
counter_switch = 0
num_switches = []
for i in range(len(df_gold)):
    answers = df_gold['ChatGPT-4 5 runs'][i]
    if len(set(answers)) > 1:
        counter_switch += 1
        num_switches.append(len(set(answers)))

In [33]:
Counter(num_switches)

Counter({2: 144, 3: 16})

#### Pretend model is 6th annotator

In [112]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\model_agreement_per_class.pkl", 'rb') as f:
    model_agreement_per_class = pickle.load(f)

In [110]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 4, "Surface contradiction": 5}

In [111]:
label_to_num = {'Unrelated': 1, "Consistent": 2,  "Inconsistent": 3, "Indirect inconsistency": 3, "Factual inconsistency": 3, "Surface contradiction": 3}

In [114]:
with open("all_sample_to_responses_some_6.pkl", 'rb') as f:
    final_sample_to_responses = pickle.load(f)

In [38]:
for el in final_sample_to_responses:
    model_label = df_gold[df_gold['Input'] == el]['ChatGPT-3.5 3 classes'].iloc[0]
    final_sample_to_responses[el].append(model_label)